In [1]:
#Import necessary packages
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder,DeployTransactionBuilder,MessageTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
import requests
import json
import pickle
import csv
import os
from repeater import retry
from iconsdk.libs.in_memory_zip import gen_deploy_data_content

In [31]:
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
SCORE_ADDRESS = "cxe02f3399b676823f7125d49e659ff748282ff8cc"
NEWTON_ADDRESS = "hx43e1f6be06a62a293d06867adc27e7acc9affe69"
DEPLOY_PARAMS = {}

In [3]:
#Start ICON service and wallet
icon_service = IconService(HTTPProvider("https://bicon.net.solidwallet.io", 3))
#icon_service = IconService(HTTPProvider("http://localhost:9000", 3))

# Returns block information by block height
block = icon_service.get_block(1000)


{'version': '0.3',
 'height': 1000,
 'signature': 'Oj2VRhXDnd/3IdrE0ClPKEgGLkYLZt/p/o7D9rlspLoSAEA1vYSZyaOoF5n1QV8hpZaObMuZBGaRm13mjYmcHgA=',
 'prev_block_hash': 'cdd0017615d2af92227ec54d3147b199ba1417e1e6ec99cf33e7005a2b709692',
 'merkle_tree_root_hash': '0000000000000000000000000000000000000000000000000000000000000000',
 'time_stamp': 1586159775591512,
 'confirmed_transaction_list': [],
 'block_hash': '95ea963d180936776a650b6b35640c20c1ca951e26a0b55cb11b1b555ecb559b',
 'peer_id': 'hxaad52424d4aec9dac7d9f6796da527f471269d2c',
 'next_leader': 'hxec79e9c1c882632688f8c8f9a07832bcabe8be8f'}

In [4]:
operator = KeyWallet.load('config/yeouido/keystores/operator', 'bgsa199@#')
prajwol = KeyWallet.load('config/yeouido/keystores/testPrajwol', 'bgsa199@#')

print(operator.get_address())
print(icon_service.get_balance(operator.get_address()))
print(icon_service.get_balance(prajwol.get_address()))

hx8b94a3792f336b71937709dae5487166c180c87a
13832157559790000000000
0


In [28]:
DEPLOY_PARAMS =  {
            "_tokenName": "TestToken",
            "_symbolName": "TK",
			"_initialSupply": 1000,
            "_decimals": 18
		}
        
deploy_transaction = DeployTransactionBuilder()\
    .from_(operator.get_address())\
    .to(GOVERNANCE_ADDRESS)\
    .nid(3)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content('ODIContracts'))\
    .params(DEPLOY_PARAMS)\
    .build()

estimate_step = icon_service.estimate_step(deploy_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(deploy_transaction, operator, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)


In [30]:
tx_result = icon_service.get_transaction_result(tx_hash)
tx_result

{'txHash': '0x54fb5520678dd1270069f8b8f13bea66cc32a557c1e1cd33e8f0072be3e7de50',
 'blockHeight': 3634531,
 'blockHash': '0x4ba12190ebfb50ecd29061cdced7d5a705d4d07d92495ce2249aea93c8ed359a',
 'txIndex': 1,
 'to': 'cx0000000000000000000000000000000000000000',
 'scoreAddress': 'cxe02f3399b676823f7125d49e659ff748282ff8cc',
 'stepUsed': 1169375800,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1169375800,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [130]:
UPDATE_PARAMS =  {
            "_tokenName": "TestToken1",
            "_symbolName": "TK1",
			"_initialSupply": 1000,
            "_decimals": 18
		}
        
deploy_transaction = DeployTransactionBuilder()\
    .from_(operator.get_address())\
    .to(SCORE_ADDRESS)\
    .nid(3)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content('ODIContracts'))\
    .params(UPDATE_PARAMS)\
    .build()

estimate_step = icon_service.estimate_step(deploy_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(deploy_transaction, operator, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)


In [131]:
tx_result = icon_service.get_transaction_result(tx_hash)
tx_result

{'txHash': '0x741c837190d74d14bcccb8c72c99f1263cf1e2b7b5942417d73e0f5281317ddd',
 'blockHeight': 3636487,
 'blockHash': '0x67b233aa820979479f0cdcc6e1678f4e2411894432f64543fd07e169f8ad465f',
 'txIndex': 1,
 'to': 'cxe02f3399b676823f7125d49e659ff748282ff8cc',
 'stepUsed': 1759851400,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1759851400,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [ ]:
external_methods = ["name", "symbol", "decimals", "totalSupply"]
for method in external_methods:
    call = CallBuilder().from_(operator.get_address())\
                    .to(SCORE_ADDRESS)\
                    .method(method)\
                    .build()
    print(icon_service.call(call))

In [63]:
transfer_transaction = CallTransactionBuilder()\
    .from_(operator.get_address())\
    .to(SCORE_ADDRESS)\
    .nid(3)\
    .nonce(100)\
    .method('transfer')\
    .params({'_to':prajwol.get_address(), '_value':12})\
    .build()

estimate_step = icon_service.estimate_step(transfer_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(transfer_transaction, operator, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)



In [65]:
tx_result = icon_service.get_transaction_result(tx_hash)
tx_result

{'txHash': '0xceba3f5d9f8c58c4ff9642f571579111066f1f12ac9d27b6026d553ebab6f2ac',
 'blockHeight': 3635135,
 'blockHash': '0x5de116d82fa738763edb14502ffeb56695e6962df8ae244569cdc031ac1c5814',
 'txIndex': 1,
 'to': 'cxe02f3399b676823f7125d49e659ff748282ff8cc',
 'stepUsed': 153920,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 153920,
 'eventLogs': [{'scoreAddress': 'cxe02f3399b676823f7125d49e659ff748282ff8cc',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'hx8b94a3792f336b71937709dae5487166c180c87a',
    'hx49763207da6a6213aaf9a0ea02f8d877fbd0105a',
    '0xc'],
   'data': ['0x4e6f6e65']}],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

In [129]:
call = CallBuilder().from_(operator.get_address())\
                    .to(SCORE_ADDRESS)\
                    .method('balanceOf')\
                    .params({'account': operator.get_address()})\
                    .build()
print(icon_service.call(call))

0x3635c9adc5ded15dbc


In [124]:
call = CallBuilder().from_(operator.get_address())\
                    .to(SCORE_ADDRESS)\
                    .method('balanceOf')\
                    .params({'account': prajwol.get_address()})\
                    .build()
print(icon_service.call(call))

0xeca2b759f


In [126]:
params={
    "value": 3235260,
}

call_transaction = CallTransactionBuilder()\
    .from_(operator.get_address())\
    .to(SCORE_ADDRESS)\
    .nid(3)\
    .nonce(100)\
    .method("mintTo")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, operator,step_limit)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash


'0x3f30ab6f7c19b491b8b3c569a85a9a6c619aede68854cfd7283cecb882545cd7'

In [128]:
tx_result = icon_service.get_transaction_result(tx_hash)
tx_result


{'txHash': '0x3f30ab6f7c19b491b8b3c569a85a9a6c619aede68854cfd7283cecb882545cd7',
 'blockHeight': 3636321,
 'blockHash': '0x66758dad273b2d1063ea45cc44902cc97598cf76ae1004f602f036e979625398',
 'txIndex': 1,
 'to': 'cxe02f3399b676823f7125d49e659ff748282ff8cc',
 'stepUsed': 141940,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 141940,
 'eventLogs': [{'scoreAddress': 'cxe02f3399b676823f7125d49e659ff748282ff8cc',
   'indexed': ['Balance(int,str)', '0x3635c9adc5dea00000'],
   'data': ['BEFORE']},
  {'scoreAddress': 'cxe02f3399b676823f7125d49e659ff748282ff8cc',
   'indexed': ['Balance(int,str)', '0x3635c9adc5ded15dbc'],
   'data': ['AFTER']}],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x